In [118]:
import sys
sys.path.append('../')

In [119]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, MetaData, Table, text

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

In [120]:
USER_DB = POSTGRES_UTEA['USER']
PASS_DB = POSTGRES_UTEA['PASSWORD']
HOST_DB = POSTGRES_UTEA['HOST']
PORT_DB = POSTGRES_UTEA['PORT']
NAME_DB = POSTGRES_UTEA['DATABASE']

ENGINE = create_engine(f'postgresql+psycopg://{USER_DB}:{PASS_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}')

PATH_OUTPUT = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\ESTIMATIVAS\V06'

metadata = MetaData()

In [121]:
# obtiene un resumen de la capa de estimativa (idd - area)
def get_resumen_estimativa():
    try:
        query = "select cast(unidad_01 as text) || '|' || cast(unidad_03 as text) || '|' || unidad_05 as idd, sum(area) as area from catastro_iag.estimativa group by idd"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de COSECHA 2025:", e)
        return pd.DataFrame()
    return None

# obtiene un resumen de la capa de estimativa (idd - area)
def get_resumen_cosecha():
    try:
        query = "select cast(unidad_01 as text) || '|' || cast(unidad_03 as text) || '|' || unidad_05 as idd, sum(area) as area from catastro_iag.cosecha_2025 group by idd"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de COSECHA 2025:", e)
        return pd.DataFrame()
    return None

def eliminar_propiedad_db(cod_prop):
    """
    Elimina registros con el cod_prop enviado de la tabla estimativa
    """

    sql_query = text(f"""
        DELETE FROM catastro_iag.estimativa
        WHERE unidad_01 = :cod_prop
    """)

    try:
        with ENGINE.connect() as conn:
            result = conn.execute(sql_query, {"cod_prop": cod_prop})
            conn.commit()  # Asegura que se guarden los cambios
        print(f"Se eliminaron {result.rowcount} registros de la propiedad: {cod_prop}")
        return result.rowcount
    except Exception as e:
        print(f"Error al ejecutar la eliminación: {e}")
        return None
    


In [123]:
resumen_estimativa = get_resumen_estimativa()
resumen_cosecha = get_resumen_cosecha()

print('Total Resumen Estimativa:', resumen_estimativa['area'].sum())
print('Total Resumen SHPCosecha:', resumen_cosecha['area'].sum())

Total Resumen Estimativa: 55357.27726918646
Total Resumen SHPCosecha: 55355.09888906363


In [124]:
# identifiar lotes que estan en ESTIMATIVAS y no en SHP_Cosecha
faltantes = resumen_estimativa[~resumen_estimativa['idd'].isin(resumen_cosecha['idd'])].copy()
if len(faltantes) > 0:
    # Dividir la columna idd en 3 columnas nuevas ('unidad_01', 'unidad_03', 'unidad_05')
    faltantes[['unidad_01', 'unidad_03', 'unidad_05']] = faltantes['idd'].str.split('|', expand=True)
    # extraer lista de unicos de codigos de propiedad
    codigos_prop = list(set(faltantes['unidad_01']))
    # convertir lista de str a int
    codigos_prop = [int(i) for i in codigos_prop]
    codigos_prop.sort()
    codigos_prop
else:
    codigos_prop = []
    print('No existen lotes que esten en ESTIMATIVAS y no en SHP COSECHA')

No existen lotes que esten en ESTIMATIVAS y no en SHP COSECHA


In [125]:
for i in codigos_prop:
    eliminar_propiedad_db(i)

In [126]:
resumen_estimativa = get_resumen_estimativa()
resumen_cosecha = get_resumen_cosecha()

print('Total Resumen Estimativa:', resumen_estimativa['area'].sum())
print('Total Resumen SHPCosecha:', resumen_cosecha['area'].sum())

Total Resumen Estimativa: 55357.27726918646
Total Resumen SHPCosecha: 55355.09888906363


In [127]:
# identifiar lotes que estan en ESTIMATIVAS y no en SHP_Cosecha
faltantes = resumen_cosecha[~resumen_cosecha['idd'].isin(resumen_estimativa['idd'])].copy()
if len(faltantes) > 0:
    # Dividir la columna idd en 3 columnas nuevas ('unidad_01', 'unidad_03', 'unidad_05')
    faltantes[['unidad_01', 'unidad_03', 'unidad_05']] = faltantes['idd'].str.split('|', expand=True)
    # extraer lista de unicos de codigos de propiedad
    codigos_prop = list(set(faltantes['unidad_01']))
    # convertir lista de str a int
    codigos_prop = [int(i) for i in codigos_prop]
    codigos_prop.sort()
    codigos_prop
else:
    codigos_prop = []
    print('No existen lotes que esten en SHP COSEHCA y no en ESTIMATIVA')

No existen lotes que esten en SHP COSEHCA y no en ESTIMATIVA


In [128]:
codigos_prop

[]

In [129]:
for i in codigos_prop:
    eliminar_propiedad_db(i)